---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
import numpy as np
from datetime import datetime

In [3]:
def date_sorter():

    # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    d1 =df.str.extractall(r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2,4})')
    d1.reset_index(inplace=True)
    d1.set_index('level_0', inplace=True)
    d1['year']=d1['year'].apply(lambda x: '19'+x if len(x)<=2 else x) # convert yy to yyyy
    
    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    d2 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[-.]* (?P<day>(?:\d{1,2}))[?:, -]*(?P<year>\d{4})')
    d2.reset_index(inplace=True)
    d2.set_index('level_0', inplace=True)
    
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    # Feb 2009; Sep 2009; Oct 2010
    d3 = df.str.extractall(r'(?P<day>(?:\d{1,2} ))?(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[?:, -]*(?P<year>\d{4})')
    d3.reset_index(inplace=True)
    d3.set_index('level_0', inplace=True)
    d3.day.fillna(1, inplace=True) # fill day missing values
    
    mapping_list ={'Apr' :4, 'April':4, 'Aug' :8, 'August': 8, 'Dec':12, 'December':12, 'Decemeber':12,
         'Feb':2, 'February':2, 'Jan':1, 'Janaury':1, 'January':1, 'Jul':7, 'July':7, 'Jun':6, 'June':6,
         'Mar':3, 'March':3, 'May':5, 'Nov':11, 'November':11, 'Oct':10, 'October':10, 'Sep':9,'September':9}
    
    
    # mapping the month to digits
    d2['month']= d2.month.map(mapping_list)
    d3['month'] = d3.month.map(mapping_list)
    
    '''
    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    d3 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* )((?:\d{1,2}(?:st|nd|rd|th), )\d{2, 4})')
    d3.reset_index(inplace=True)
    d3.set_index('level_0', inplace=True)
    '''
    
    # 6/2008; 12/2009
    d4 = df.str.extractall(r'(?P<month>\d{1,2})[/](?P<year>\d{4})')
    d4.reset_index(inplace=True)
    d4.set_index('level_0', inplace=True)
    
    # 2009; 2010
    d5 = df.str.extractall(r'[a-z]?[^0-9](?P<year>\d{4})[^0-9]')
    d6 = df.str.extractall(r'^(?P<year>\d{4})[^0-9]')
    d5 = d5.append(d6)
    d5.reset_index(inplace=True)
    d5.set_index('level_0', inplace=True)

    
    # remove common level_0
    index_0 = [i for i in d4.index if (i not in d1.index)]
    d4 = d4.loc[index_0]
    d4['day'] = 1
    
    index_1 = [i for i in d5.index if (i not in d1.index) and (i not in d2.index) and (i not in d3.index) and (i not in d4.index)]
    d5 = d5.loc[index_1]
    d5['month'] = 1
    d5['day'] = 1
    
    # concat
    d1 = d1[['month', 'day', 'year']]
    d2 = d2[['month', 'day', 'year']]
    d3 = d3[['month', 'day', 'year']]
    d4 = d4[['month', 'day', 'year']]
    d5 = d5[['month', 'day', 'year']]
    result = pd.concat([d1,d2,d3,d4,d5])
    
    result = result[result.day.apply(int) <= 31] # drop days
    result = result[result.year.apply(int) <= 5999] # drop years
    result = result[result.month.apply(int) <= 12] # drop month

    
    result['date'] = pd.to_datetime(result['month'].apply(str)+'/'+result['day'].apply(str)+'/'+ result['year'].apply(str))
    result.sort_index(inplace=True)
    
    answer = pd.Series(result.sort_values(by='date').index)[:500]
    return answer

In [4]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64